# Using Intelligence from Indeed Job postings to augment my skills.

## 1. Problem statement
I am confused on what skills are most relevant in the current job market. So after learning Ensemble learning, I loved the concept and thought of applying that idea to find my skills gap using ensemble of relevant skill data from the Indeed job posting. More like an wisdom of crowd approach. So I am aiming to find relevant skills that I should learn next before applying for my next Job to improve my odds of getting hired, and feel more confident.

## 2. Approach
First we will download data from Indeed. I made a web scraper https://github.com/rajdeepmondal1/indeed-job-scraper that scrapes the https://in.indeed.com/ website for our essential data after searching for Data Scientist, Data Engineer, Data Science Consultant, Data Analyst and Machine Learning Engineer. Since the lines are blurry about what a Data Scientist does, we scrape data about these adjacent titles to remove bias in our data. Indeed website offers millions of searchable job listings, posted by understaffed employers. It has a built-in search engine, which allows us to filter the jobs by keyword. We will download the required information and store it in separate html files for the job listings.

Our goal is to extract common data science and machine learning skills from the downloaded data. We’ll then compare these skills to our skills to determine which skills are missing. To reach our goal, we’ll proceed like this:
 - Parse out all the text from the downloaded HTML files.
 - Explore the parsed output to learn how job skills are commonly described in online postings. Perhaps specific HTML tags are more commonly used to underscore job skills.
 - Try to filter out any irrelevant job postings from our dataset. The search engine isn’t perfect. Perhaps some irrelevant postings were erroneously downloaded. We can evaluate relevance by comparing the postings with our skill list.
 - Cluster the job skills within the relevant postings, and visualize the clusters.
 - Make a plan to improve on the skills that are missing.

In [1]:
import zipfile

def unzip_data(filename):
  """
  Unzips filename into the current working directory.
  Args:
    filename (str): a filepath to a target zip folder to be unzipped.
  """
  zip_ref = zipfile.ZipFile(filename, "r")
  zip_ref.extractall()
  zip_ref.close()

In [5]:
!git clone https://github.com/rajdeepmondal1/what-skills-to-augment-my-arsenal-main

Cloning into 'what-skills-to-augment-my-arsenal-main'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 10 (delta 1), reused 6 (delta 0), pack-reused 0
Unpacking objects: 100% (10/10), done.


In [6]:
data_location = '/content/what-skills-to-augment-my-arsenal-main/data/data_2161_listings.zip'

In [7]:
unzip_data(data_location)

## 3. Dataset description
I have scraped a total of 2161 listings using https://github.com/rajdeepmondal1/indeed-job-scraper from https://in.indeed.com/ including -
 - Data Scientist with 735 listings
 - Machine Learning Engineer with 654 listings
 - Data Engineer with 317 listings
 - Data Science Consultant with 151 listings
 - Data Analyst with 304 listings
The title, company name, ratings and job description was scraped from the website for each listing.

Our rough draft of skills are stored in the skills.txt file.

We will proceed as follows:

1. Parse all text from the downloaded HTML files.
2. Explore the parsed output to learn how job skills are described in online postings. We’ll pay particular attention to whether certain HTML tags are more associated with skill descriptions.
3. Attempt to filter any irrelevant job postings from our dataset.
4. Cluster job skills based on text similarity.
5. Visualize the clusters using word clouds.
6. Adjust clustering parameters, if necessary, to improve the visualized output.
7. Compare the clustered skills to our resume to uncover missing skills.

## 4. Extracting skill requirements from job posting data

### 4.1. Loading HTML files

In [17]:
import glob

# Evading any warnings
import warnings
warnings.filterwarnings('ignore')

In [18]:
html_contents = []

for file_name in sorted(glob.glob('/content/data_2161_listings/*/*.html')):
    with open(file_name, encoding="utf8") as f:
        html_contents.append(f.read())
        
print(f"We've loaded {len(html_contents)} HTML files.")

We've loaded 2161 HTML files.


### 4.2. Parsing HTML files

In [19]:
# import beautiful soup
from bs4 import BeautifulSoup as bs

In [20]:
soup_objects = []

# parse through all the html file and store it in soup objects
for html in html_contents:
    soup = bs(html)
    soup_objects.append(soup)

### 4.3. Extracting Title, Company Name and Description into their own variables.

In [21]:
# import pandas
import pandas as pd

In [22]:
# dictionary to store values of title, company name and description
html_dict = {
    'title': [],
    'company_name': [],
    'description': []
    }

for soup in soup_objects:
    try:
        # try to get the title using the class tag 'jobsearch-JobInfoHeader-title'
        title = soup.findAll('h1', class_='jobsearch-JobInfoHeader-title')[0].text.strip()
    except:
        title = None  
        
    try:
        # try to get the company rating using the class tag 'jobsearch-InlineCompanyRating'
        company_temp = soup.findAll('div', class_='jobsearch-InlineCompanyRating')[0]
    except:
        pass
    
    try:
        # try to get the company name
        company_name = company_temp.findAll('a')[0].text.strip()
    except:
        company_name = None
        
    # .decompose() removes a tag from the tree, then completely destroys it and its contents
    soup.findAll('h1', class_='jobsearch-JobInfoHeader-title')[0].decompose()
    soup.findAll('div', class_='jobsearch-InlineCompanyRating')[0].decompose()
    # after the decompose function removes these elements we can take the rest as description
    description = soup.text.strip()
    html_dict['title'].append(title)
    html_dict['company_name'].append(company_name)
    html_dict['description'].append(description)
    
df_jobs = pd.DataFrame(html_dict)
df_jobs[['title', 'company_name']].describe()

,title,company_name
count,2161,1270
unique,979,404
top,Data Scientist,IBM
freq,204,37


There are 979 Unique Job Titles with Data Scientist being the most used title with a frequency of 204 and 404 Unique Company Names with the company with the most listings being IBM with 37 listings.

### 4.4. Checking for Null Values.

In [23]:
df_jobs['title'].isna().sum()

0

There are no null values in the title column, so we can proceed with our analysis as title is really important to analyze the job listings.

### 4.5. An Example Listing.

In [24]:
from IPython.core.display import display, HTML
display(HTML(html_contents[991]))

**This was a listing we extracted from the https://in.indeed.com/ website after searching for Data Scientist in India.**

The rendered job posting with a title, company name, location, brief position overview, responsibilities and quanifications. From this, we learn that the job entails surfacing insights from the data and provide strategic recommendations to better customize marketing efforts to the new region.

The skills are further elaborated in the two bolded subsections: Responsibilities, Basic Qualifications and Preferred Qualifications. Each subsection is composed of multiple single-sentence bullet points. The bullets are varied in their content but it’s worth noting that the qualifications are not that different from the responsibilities.

The qualifications focus on tools and concepts, while the responsibilities are more attuned to actions on the jobs; but in a way, their bullet points are interchangeable. Each bullet describes a skill that an applicant must have to perform well at the job. Thus, we can subdivide html_contents[0] into two conceptually different parts:

 - An initial job summary.
 - A list of bulleted skills required to get the job.

Now let us select another Random job posting to check if the Postings have a similar format.

In [26]:
display(HTML(html_contents[777]))

This job posting is for a Staff Data Scientist position at myntra.com in Bengaluru, Karnataka. The structure is similar to the previous posting. The job is summarized in the post’s initial paragraph, and then the required skills are presented in bullet points. These bulleted skills are varied in terms of technical requirements and details.

The bullet points in listing 991 and in listing 777 serve a singular purpose: they offer us brief, sentence-length descriptions of unique skills required for each position.

We will extract the bullets from each of our parsed HTML files to find out if the bullet skill descriptions appear in other job posts.

### 4.6. Extracting bullets from the HTML

In [27]:
# bullet.text.strip() Strips the line break from each bullet to avoid printing 
# the line breaks in our later investigations
df_jobs['Bullets'] = [[bullet.text.strip() for bullet in soup.find_all('li')] for soup in soup_objects]

The bullets in each job posting are stored in Bullets of the df_jobs dataframe. However, it is possible that some of the postings don’t include any bullets.

### 4.7. Measuring the percent of bulleted postings

In [28]:
bulleted_post_count = 0
for bullet_list in df_jobs['Bullets']:
    if bullet_list:
        bulleted_post_count += 1
percent_bulleted = 100 * bulleted_post_count / df_jobs.shape[0]
print(f"{percent_bulleted:.2f}% of the postings contain bullets")

86.12% of the postings contain bullets


To understand the contents of the bullet points we can print the top-ranked words in their text. We can rank these words by carrying out the ranking using term frequency inverse document frequency (TFIDF) values rather than raw counts. TFIDF rankings are less likely to contain irrelevant words.

Next, we rank the words using summed TFIDF values. First we compute a TFIDF matrix in which rows correspond to individual bullets. Then we sum across the rows of the matrix: these sums are used to rank the words, which correspond to matrix columns. Finally, we check the top five ranked words for skill-related terminology.

### 4.8. Examining the top-ranked words in the bullets.

In [29]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

In [32]:
def rank_words(text_list):
    """
      Returns a sorted Pandas table of top-ranked words
      args: text_list: input list of text
    """
    # initialize the tfidf vectorizer using stop words in english
    vectorizer = TfidfVectorizer(stop_words='english')
    # fit the vectorizer to the input text list and convert it to an array
    tfidf_matrix = vectorizer.fit_transform(text_list).toarray()
    # create a dataframe to store the words with their tfidf scores
    df = pd.DataFrame({'Words': vectorizer.get_feature_names(),
                       'Summed TFIDF': tfidf_matrix.sum(axis=0)})
    # sort the dataframe according to the tfidf scores
    sorted_df = df.sort_values('Summed TFIDF', ascending=False)
    return sorted_df

In [33]:
all_bullets = []

# store all the bullets in a list
for bullet_list in df_jobs['Bullets']:
    all_bullets.extend(bullet_list)

# rank all the words from the bullets
sorted_df = rank_words(all_bullets)
print(sorted_df[:5].to_string(index=False))

     Words  Summed TFIDF
      data   1268.549381
experience   1114.200969
  learning    585.266786
      work    565.704799
    skills    565.142082


Words like data, experience, learning, work and skills appear among the top five bulleted words. There’s reasonable evidence that the bullets correspond to individual job skills.

Next we will check how the bulleted words compare to the remaining words in each job posting by iterating over the body of each posting and delete any bulleted lists using Beautiful Soup’s decompose method. Then we extract the remaining body text and store it in a non_bullets list. Finally, we apply our rank_words function to that list and display the top five non-bullet words.

### 4.9. Examining the top-ranked words in the the description except the bullets.

In [35]:
non_bullets = []
for soup in soup_objects:
    body = soup.body
    for tag in body.find_all('li'):
        tag.decompose()
    non_bullets.append(body.text)
    
sorted_df = rank_words(non_bullets)
print(sorted_df[:5].to_string(index=False))

     Words  Summed TFIDF
      data    153.474594
experience     81.848449
       job     70.631671
      work     70.048442
    skills     63.047988


The non bulleted text seems to contain more of things like experiance, job, work, data and skills. These seem to be quite similar corrosponding to the bulleted materails.

Next we will check how the percentage of titles that has Data Scientist mentioned in their names.

### 4.10. Checking titles for references to data science positions.

In [36]:
from sklearn.utils import shuffle

In [37]:
regex = r'Data Scien(ce|tist)'
df_non_ds_jobs = df_jobs[~df_jobs.title.str.contains(regex, case=False)]

percent_non_ds = 100 * df_non_ds_jobs.shape[0] / df_jobs.shape[0]
print(f"{percent_non_ds:.2f}% of the job posting titles do not mention a data science position.")

68.90% of the job posting titles do not mention a data science position.


In [38]:
# Let's shuffle the df_non_ds_jobs DataFrame
df_non_ds_jobs = shuffle(df_non_ds_jobs)
df_non_ds_jobs.reset_index(inplace=True, drop=True)

print('These are the samples of such titles: - ')
for title in df_non_ds_jobs.title[:15]:
    print(title)

These are the samples of such titles: - 
Data Engineer - Azure
Engineer - CFD
Associate Data Engineer
Data Engineer
Associate Software Engineer
Senior Associate - Machine Learning Engineer
Engineer IIOT
Cloud Engineer in Machine Learning area
Sr. Machine Learning Engineer, SMAI
Solution Consultant - MDM
2A_Business Analyst Trainee_SQL
ETL Principal Software Engineer
Machine Learning Engineer (Remote)
Data Analyst (UK Pathways Admissions)
Quantitative Engineer - Eqs Sts Strats


68.90% of the job posting titles do not mention a data science position. However, from our sampled output, we can glean the alternative language that can be used to describe a data science job. A posting may call for a Machine Learning Engineer, a AI/ML Engineer, or a Python NLP Developer.

We will Obtain relevant job postings that partially match our existing skill set and examine which bullet points in these postings are missing from our existing skill set. With this strategy in mind, we’ll now filter the jobs by relevance.